In [1]:
import cv2
import numpy as np
import math
from skimage.draw import rectangle, rectangle_perimeter
from skimage.morphology import binary_erosion, binary_dilation, binary_closing
from skimage.transform import resize

In [2]:
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [4]:
video_capture = cv2.VideoCapture(0)
count = 0
while(1):
    count += 1
    try:  #an error comes if it does not find anything in window as it cannot find contour of max area
              #therefore this try error statement

        ret, frame = video_capture.read()
        frame=cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)

        #define region of interest
        roi=frame[100:300, 100:300]


        cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([30,255,255], dtype=np.uint8)

     #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)

    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask, kernel, iterations = 2)
        mask = cv2.erode(mask, kernel, iterations = 2)
        mask = cv2.dilate(mask, kernel, iterations = 2)
        mask = cv2.erode(mask, kernel, iterations = 2)
        mask = cv2.dilate(mask, kernel, iterations = 2)
        mask = cv2.erode(mask, kernel, iterations = 2)

    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 

        
        
        
        ## code starts here
        
    #find contours
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #find bouding box
        Xmax = max(cnt[:,0,0])
        Ymax = max(cnt[:,0,1])
        Xmin = min(cnt[:,0,0])
        Ymin = min(cnt[:,0,1])

        box = [Xmin, Ymin, Xmax, Ymax]

        boxWidth = Xmax - Xmin
        boxHeight = Ymax - Ymin

        boxarea = boxHeight * boxWidth

        boxAspectRatio = boxHeight / boxWidth

    #find orientation
        orientation = 'V'
        if(boxAspectRatio <= 1):
            orientation = 'H'
            
    #left half
        leftHalfCnt = cnt[(cnt[:,0,0] <= box[0]+boxWidth/2)]
        leftHalfCntArea = cv2.contourArea(leftHalfCnt)
    
    #right half
        rightHalfCnt = cnt[(cnt[:,0,0] > box[0]+boxWidth/2)]
        rightHalfCntArea = cv2.contourArea(rightHalfCnt)
    
    
#     #resizing contour
#         windowCenter = (Xmin + boxWidth // 2, Ymin + boxHeight // 2)
#         windowSide = 0
#         if(boxHeight > boxWidth):
#             windowSide = boxHeight
#         else:
#             windowSide = boxWidth
        
        
#         window = mask[windowCenter[0] - windowSide // 2 : windowCenter[0] + windowSide // 2,
#                       windowCenter[1] - windowSide // 2 : windowCenter[1] + windowSide // 2]
#         window = resize(window, (128, 128))

    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)


    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
    #vertical Fingers Lines
        vLines = []
        vPoints = []
        vAnchor = np.array([(Xmax + Xmin) / 2, Ymax])
        
#         if(count % 100 == 0):
        lastPoint = hull[0,:,:]
        for i in hull:
#             a = math.sqrt((lastPoint[0,0] - i[0,0])**2 + (lastPoint[0,1] - i[0,1])**2)
#             b = math.sqrt((vAnchor[0] - i[0,0])**2 + (vAnchor[1] - i[0,1])**2)
#             c = math.sqrt((lastPoint[0,0] - vAnchor[0])**2 + (lastPoint[0,1] - vAnchor[1])**2)
#             s = (a + b + c) / 2
#             ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
#             # distance 
#             if(a != 0):
#                 d=(2*ar)/a
#             else:
#                 d = 0;
                
#             # apply cosine rule here
#             if(2*b*c != 0):
#                 angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
#             else:
#                 angle = 0
#                 print(i[0,:])
#                 print(vAnchor)
            if(dist(i[0,:], vAnchor) > 80 and np.absolute(i[0,0] - lastPoint[0,0]) > 10):
                
#                 if(count % 100 == 0): print(dist(i[0,:],vAnchor));
#                 print(lastPoint[0,0] - i[0,0])
#                 cv2.circle(roi, (i[0, 0], i[0, 1]), 3, [255,255,255], -1)
                vPoints.append((i[0, 0], i[0, 1]))
                vLines.append([(int(i[0,0]),int(i[0,1])), (int(vAnchor[0]), int(vAnchor[1]))])
                lastPoint = i
        
       
    #horizontal fingers lines
        hLines = []
        hPoints = []
        hAnchor = np.array([Xmax, (Ymax + Ymin) / 2])
                           
#         if(count % 100 == 0):
        lastPoint = hull[0,:,:]
        for i in hull:
#                 print(i[0,:])
#                 print(vAnchor)
            if(dist(i[0,:],hAnchor) > 80 and dist(i[0,:], lastPoint[0,:]) > 10):        
#                 if(count % 100 == 0): print(dist(i[0,:],vAnchor));
#                 print(lastPoint[0,0] - i[0,0])
#                 cv2.circle(roi, (i[0, 0], i[0, 1]), 3, [255,255,255], -1)
                hPoints.append((i[0, 0], i[0, 1]))
                hLines.append([(int(i[0,0]),int(i[0,1])), (int(hAnchor[0]), int(hAnchor[1]))])
                lastPoint = i
   
    # drawing fingers lines depending on orientation
#         if(orientation == 'V'):
#             for i in vLines:
#                 cv2.line(roi, i[0], i[1], [0,255,0], 2)
#             for i in vPoints:
#                 cv2.circle(roi, i, 3, [255,255,255], -1)
#         else:
#             for i in hLines:
#                 cv2.line(roi, i[0], i[1], [0,0,255], 2)
#             for i in hPoints:
#                 cv2.circle(roi, i, 3, [255,255,255], -1)

        
     #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)

    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100

     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)

        contourRatio = areacnt / boxarea

    # no. of defects in each direction
        upFingers = 0
        downFingers = 0
        leftFingers = 0
        leftFingersRight = 0
        rightFingersAcute = 0
        rightFingersRight = 0

    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)

            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

            #distance between point and convex hull
            d=(2*ar)/a

            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

            if (far[1] > start[1]) & (far[1] > end[1]) & (far[0] < start[0]) & (far[0] > end[0]): 
                if (angle <= 70 and d > 30):
                    upFingers += 1
                    cv2.circle(roi, far, 3, [255,0,0], -1)
            elif (far[0] > start[0]) & (far[0] > end[0]) & (far[1] > start[1]) & (far[1] < end[1]):
                if (angle <= 50 and d > 30):
                    leftFingers += 1
                    cv2.circle(roi, far, 3, [0,0,0], -1)
                elif (angle >= 50 and angle <=120 and d > 30):
                    leftFingersRight += 1
                    cv2.circle(roi, far, 3, [100,100,100], -1)
            elif (far[1] < start[1]) & (far[1] < end[1]) & (far[0] > start[0]) & (far[0] < end[0]):
                if (angle <= 70 and d > 30):
                    downFingers += 1
                    cv2.circle(roi, far, 3, [0,255,0], -1)
            elif (far[0] < start[0]) & (far[0] < end[0]):
                if (angle >= 70 and angle <= 120 and d > 30):
                    rightFingersRight += 1
                    cv2.circle(roi, far, 3, [0,0,255], -1)
                elif (angle <= 70 and d > 30):
                    rightFingersAcute += 1
                    cv2.circle(roi, far, 3, [255, 0, 255], -1)

            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)

        upFingers += 1
        downFingers += 1
        leftFingers += 1
        leftFingersRight += 1
        rightFingersAcute += 1
        rightFingersRight += 1

        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        currLetter = ''
        if upFingers == 1 and downFingers == 1 and leftFingers == 1 and leftFingersRight == 1 and rightFingersAcute == 1 and rightFingersRight == 1:
            if areacnt<2000:
                 currLetter = 'Put hand in the box'
            else:
                if arearatio < 8.5: 
                    # no fingers
                    if boxAspectRatio < 1.5:
                        currLetter = 'A'
                    else:
                        currLetter = 'B'
                else: 
                    # one finger
                    if orientation == 'V':
                        if leftHalfCntArea < rightHalfCntArea:
                            currLetter = 'D'
                        else:
                            currLetter = 'I'
                    elif orientation == 'H' and leftHalfCntArea < rightHalfCntArea:
                        currLetter = 'G'
#                 if len(vLines) == 0:
#                     currLetter ='A'
#                 elif len(vLines) == 1:
#                     if leftHalfCntArea < rightHalfCntArea:
#                         currLetter = 'D'
#                     else:
#                         currLetter = 'I'
#                 elif len(vLines) == 2:
#                     currLetter ='U'
#                 elif len(vLines) == 3:
#                     currLetter ='F'
#                 elif len(vLines) == 4:
#                     currLetter ='B'
                 
        elif upFingers == 2 and (rightFingersRight == 2 or rightFingersAcute == 2):
            currLetter = '3'
        
        elif upFingers == 2:
            currLetter = 'V'
        
        elif (upFingers == 4 and (rightFingersAcute == 2 or rightFingersRight == 2)) or (upFingers == 3 and ((rightFingersAcute ==2 and rightFingersRight == 2) or rightFingersAcute == 3)):
            currLetter = '5'
        
        elif upFingers == 4 or (upFingers == 3 and rightFingersAcute == 2):
            currLetter = '4'
        
        elif upFingers == 3:
            currLetter = 'W'
           
        elif rightFingersAcute == 2:
            currLetter = 'C'
            
        elif rightFingersRight == 2:
            currLetter = 'L'
            
        elif leftFingers == 3:
            currLetter = 'E'
        
        elif leftFingersRight == 2:
            currLetter = 'P'
        
        elif downFingers == 2:
            currLetter = 'N'
        
        elif downFingers == 3:
            currLetter = 'M'
        
        else:
            currLetter = 'Reposition'

        cv2.putText(frame,currLetter,(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    
        
        # show the windows
        cv2.imshow('mask', mask)
#         cv2.imshow('window', window)
        cv2.imshow('frame', frame)
    
    except Exception as e:
        print(e)
#         pass
    

    # exit condition
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
video_capture.release()    
    

[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 65 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 65 160]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 66 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 64 160]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 66 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use

[ 66 158]
[ 66 158]
[ 67 158]
[ 66 158]
[ 66 158]
[120 160]
[120 160]
[ 66 158]
[ 66 158]
[ 66 158]
[ 66 158]
[ 69 158]
[ 66 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 66 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 68 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 68 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 64 160]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 65 160]
The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[ 67 158]
[ 68 158]
[ 66 158]
[ 65 160]
[ 66 158]
The truth valu

KeyboardInterrupt: 